# Scratch

### Sequence length

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
test = pd.read_csv('/scratch/gpfs/aa8417/QCB557_project/data/test.csv')
train = pd.read_csv('/scratch/gpfs/aa8417/QCB557_project/data/train.csv')

In [ ]:
test_seqs = test['sequence'].tolist()
train_seqs = train['sequence'].tolist()

In [ ]:
seqs = test_seqs + train_seqs

In [ ]:
def get_seq_lengths(sequences):
    seq_lengths = []
    for sequence in sequences:
        length = len(sequence)
        seq_lengths.append(length)
    return seq_lengths

lengths = get_seq_lengths(seqs)

In [ ]:
plt.hist(lengths, bins=20)
plt.xlabel('length of DNA sequences in train and test set')
plt.ylabel('frequency')
plt.title('histogram of DNA sequence lengths')

plt.show()

In [ ]:
min(lengths)

In [ ]:
max(lengths)

So, sequence lengths range from 290 to 500. So max_length in tokenizer should be 500.

### Dataset sizes

In [2]:
import pandas as pd
import numpy as np

In [10]:
histones = ['H3', 'H3K14ac', 'H3K36me3', 'H3K4me1', 'H3K4me2', 'H3K4me3', 'H3K9ac', 'H4', 'H4ac']

percents_0 = []
percents_1 = []
totals = []

data_dict = {}


for i in range(9):
    histone = histones[i]
    train = pd.read_csv(f'/scratch/gpfs/aa8417/QCB557_project/data/{histone}/train.csv')
    test = pd.read_csv(f'/scratch/gpfs/aa8417/QCB557_project/data/{histone}/test.csv')
    
    df = pd.concat([train, test], ignore_index=True)
    
    label_counts = df['label'].value_counts(normalize=True) * 100
    percent_0 = label_counts[0]
    percent_1 = label_counts[1]

    rows, _ = df.shape
    
    percents_0.append(percent_0)
    percents_1.append(percent_1)
    totals.append(rows)

In [11]:
percents_0

[48.76712328767123,
 43.200798838053736,
 45.83715596330275,
 45.493575780534776,
 40.86953687709807,
 46.726813228620344,
 44.514433806061476,
 55.61947811793713,
 46.00381287578824]

In [12]:
percents_1

[51.23287671232877,
 56.79920116194626,
 54.162844036697244,
 54.50642421946523,
 59.13046312290193,
 53.273186771379656,
 55.48556619393852,
 44.38052188206287,
 53.99618712421176]

In [14]:
histones

['H3',
 'H3K14ac',
 'H3K36me3',
 'H3K4me1',
 'H3K4me2',
 'H3K4me3',
 'H3K9ac',
 'H4',
 'H4ac']

In [13]:
totals

[14965, 33048, 34880, 31677, 30683, 36799, 27782, 14601, 34095]

### Code bank

In [ ]:
#DON'T NEED THIS ANYMORE
#change classification head of the model
#reference: https://discuss.huggingface.co/t/how-do-i-change-the-classification-head-of-a-model/4720/3
class BinaryDNABERT2Model(nn.Module):
    def __init__(self):
        super(BinaryDNABERT2Model, self).__init__()

        self.base_model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True, config=config).to(device)
        self.dropout = nn.Dropout(0.5)
        self.linear = nn.Linear(768, 2) # output features from bert is 768 and 2 is number of labels
        
    def forward(self, input_ids, attn_mask):
        outputs = self.base_model(input_ids, attention_mask=attn_mask)
        # You write you new head here
        outputs = self.dropout(outputs[0])
        outputs = self.linear(outputs)
        
        return outputs

In [ ]:
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.Tensor(labels).long()
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

## Attention weight stuff

In [1]:
import pandas as pd
import torch
from transformers import (
    AutoTokenizer, 
    AutoModel, 
    AutoModelForSequenceClassification, 
    DataCollatorForTokenClassification, 
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer, 
    DataCollatorForLanguageModeling
)
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
from transformers.models.bert.configuration_bert import BertConfig 
import numpy as np
from datasets import load_dataset, Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
#from training_args_module import training_args
import transformers
import os

import csv
from Bio import SeqIO


import argparse

/home/aa8417/.conda/envs/dna/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
princeton_id = 'aa8417'
project_dir = f'/scratch/gpfs/{princeton_id}/QCB557_project'

model_name = 'fine_tune_new_v3'
model_out_dir = f'{project_dir}/models/{model_name}'

# use gpu
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [7]:
config = BertConfig.from_pretrained(f'{model_out_dir}/config.json', output_attentions=True)
print(config.num_labels) #2 labels
model = AutoModelForSequenceClassification.from_pretrained(model_out_dir, trust_remote_code=True, config=config)
model.to(device)
'''
config = BertConfig.from_pretrained("zhihan1996/DNABERT-2-117M", output_hidden_states=True, output_attentions=True)
print(config.num_labels) #2 labels
model = AutoModelForSequenceClassification.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True, config=config)
model.to(device)
'''
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True, padding=True)
tokenizer.pad_token = "X"

2


In [8]:
assert config.output_attentions == True

In [9]:
sequence = 'TCAATCACTCTAAGAAGAAACAATCTAGAAAACTAATTAAATCGAGAAGACAAGATTTCACCACTAATGTAGAAATAAACCCTATTAACAAGAACTTGTATTTCCCACACATTTCTGCTGTTCAAATTTTTGACTTCTATAAAAATGAGCAAGTTAACTATCAGTATTTAACATCAGGTGTCAACAATTCTATGGGTAAAGTTAGATTTGAACTGAATTTACAAGACCCAATAATAACTGATTTGAAGTTCACCAAAGATGGGCAATGGATGATTACATACGAAATTGAGTATCCGCCAAATGACCTCTTATCTTCCAAGGACTTAACTCATATCTTGAAATTTTGGACCAAAAACGATAATGAGACAAATTGGAATTTGAAAACGAAAGTAATAAATCCACACGGGATAAGTGTCCCAATTACCAAGATATTGCCTTCACCAAGATCAGTTAATAATAGTCAAGGCTGTTTAACGGCTGACAACAACGGTGGACTGAAA'

In [12]:
def get_model_output(model, tokenizer, sequence, device):
    inputs = tokenizer(sequence, padding='max_length', max_length=128, return_tensors='pt').to(device)
    input_ids = inputs['input_ids'].to(device)
    #print(input_ids)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, output_attentions=True)
        print(outputs.hidden_states)

    predicted_probs = torch.softmax(outputs.logits, dim=-1)
    attention_weights = outputs.attentions
    print(np.array(attention_weights).shape)

    predicted_label = torch.argmax(predicted_probs, dim=-1).item()

    return predicted_label, predicted_probs, attention_weights

predicted_label, predicted_probs, attention_weights = get_model_output(model, tokenizer, sequence, device)
print("Predicted Label:", predicted_label)
print("Predicted Probabilities:", predicted_probs)
print("Attention Weights:", attention_weights)


tensor([[[ 0.1051, -0.0844, -0.0708,  ..., -0.1008,  0.3531, -0.0425],
         [-0.0933,  0.1621, -0.1887,  ...,  0.3297,  0.3745,  0.6088],
         [ 0.1130,  0.3378,  0.1361,  ..., -0.0580, -0.0935,  0.5449],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
       device='cuda:0')
()
Predicted Label: 0
Predicted Probabilities: tensor([[9.9968e-01, 3.2139e-04]], device='cuda:0')
Attention Weights: None


In [ ]:
config.output_attentions

In [ ]:
print(model)

In [ ]:
outputs.hidden_state

In [7]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4096, 768, padding_idx=0)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertUnpadAttention(
            (self): BertUnpadSelfAttention(
              (dropout): Dropout(p=0.0, inplace=False)
              (Wqkv): Linear(in_features=768, out_features=2304, bias=True)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (mlp): BertGatedLinearUnitMLP(
            (gated_layers): Linear(in_features=768

In [ ]:
idntity = testing_indx_0/1
sequence
input_ids
attention_weights
embedding 